In [36]:
import pandas as pd
import sqlite3

## • create a connection to the database using the library sqlite3

In [37]:
con = sqlite3.connect('../data/checking-logs.sqlite.sqlite copy')

## • get the schema of the table test

In [38]:
table_test = 'test'
query = f'PRAGMA table_info({table_test});'
pd.io.sql.read_sql(query, con, index_col='cid')

,name,type,notnull,dflt_value,pk
cid,,,,,
0,index,INTEGER,0,None,0
1,uid,TEXT,0,None,0
2,labname,TEXT,0,None,0
3,first_commit_ts,TIMESTAMP,0,None,0
4,first_view_ts,TIMESTAMP,0,None,0


## • get only the first 10 rows of the table checker to check what the table looks like

In [39]:
count = 10
firstRows = f'SELECT * FROM {table_test} LIMIT {count};'
pd.io.sql.read_sql(firstRows, con, index_col='index')

,uid,labname,first_commit_ts,first_view_ts
index,,,,
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035


In [40]:
table_deadlines = 'deadlines'
query = f'PRAGMA table_info({table_deadlines});'
pd.io.sql.read_sql(query, con)


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


## be sure to use parse_date to parse as dates.

In [58]:
count = 10
firstRows = f'SELECT * FROM {table_deadlines} LIMIT {count};'
pd.io.sql.read_sql(firstRows, con, index_col='index', parse_dates=[table_deadlines])

,labs,deadlines
index,,
0,laba04,2020-04-26 23:59:59
1,laba04s,2020-04-26 23:59:59
2,laba05,2020-05-03 23:59:59
4,laba06,2020-05-24 23:59:59
5,laba06s,2020-05-24 23:59:59
3,project1,2020-05-16 23:59:59


[Absolute Delta](https://sciencing.com/calculate-delta-between-two-numbers-5893964.html)

[Разница между двумя датами в Sqlite3](https://coderoad.ru/55543509/%D0%A0%D0%B0%D0%B7%D0%BD%D0%B8%D1%86%D0%B0-%D0%BC%D0%B5%D0%B6%D0%B4%D1%83-%D0%B4%D0%B2%D1%83%D0%BC%D1%8F-%D0%B4%D0%B0%D1%82%D0%B0%D0%BC%D0%B8-%D0%B2-Sqlite3)

[Разница между двумя датами в SQLite](https://overcoder.net/q/1305/%D1%80%D0%B0%D0%B7%D0%BD%D0%B8%D1%86%D0%B0-%D0%BC%D0%B5%D0%B6%D0%B4%D1%83-%D0%B4%D0%B2%D1%83%D0%BC%D1%8F-%D0%B4%D0%B0%D1%82%D0%B0%D0%BC%D0%B8-%D0%B2-sqlite)

[Difference between 2 dates in SQLite](https://stackoverflow.com/questions/289680/difference-between-2-dates-in-sqlite)

[SQLite: julianday Function](https://www.techonthenet.com/sqlite/functions/julianday.php)


• find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query

◦ do this by joining the table with the table deadlines

◦ the difference should be displayed in hours

◦ do not take the lab ’project1’ into account, it has longer deadlines and will be an outlier

◦ the value should be stored in the dataframe df_min with the corresponding uid

In [100]:
aggregated_query_min = """
SELECT  uid,
        MIN (
                CAST (
                        (julianday(test.first_commit_ts) - julianday(DATETIME(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER
                     )
            ) AS delta
FROM test LEFT JOIN deadlines
ON test.labname = deadlines.labs
WHERE NOT test.labname = 'project1'
"""

df_min = pd.io.sql.read_sql(aggregated_query_min, con)
df_min


,uid,delta
0,user_30,-202


• do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [101]:
aggregated_query_max = """
SELECT  uid,
        MAX (
                CAST (
                        (julianday(test.first_commit_ts) - julianday(DATETIME(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER
                     )
            ) AS delta
FROM test LEFT JOIN deadlines
ON test.labname = deadlines.labs
WHERE NOT test.labname = 'project1'
"""

df_max = pd.io.sql.read_sql(aggregated_query_max, con)
df_max

,uid,delta
0,user_25,-2


• do the samething but for the average,using only one query,this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [102]:
aggregated_query_average = """
SELECT  AVG (
                CAST (
                        (julianday(test.first_commit_ts) - julianday(DATETIME(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER
                     )
            ) AS delta
FROM test LEFT JOIN deadlines
ON test.labname = deadlines.labs
WHERE NOT test.labname = 'project1'
"""

df_average = pd.io.sql.read_sql(aggregated_query_average, con)
df_average


,delta
0,-89.125


 we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference

◦ using only one query, create a table with the columns: uid, avg_diff, pageviews

◦ uid is the uids that exist in the test

◦ avg_diff is the average delta between the first commit and the lab deadline per user

◦ pageviews is the number of Newsfeed visits per user

◦ do not take the lab ’project1’ into account

◦ store it to the dataframe views_diff

◦ use the Pandas method corr() to calculate the correlation coefficient between the number of pageviews and the difference

In [107]:
aggregated_query_test_the_hypothesis = """
SELECT  test.uid,
        AVG (
                CAST (
                        (julianday(test.first_commit_ts) - julianday(DATETIME(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER
                     )
            ) AS delta,
        pageviews
FROM test
LEFT JOIN deadlines ON test.labname = deadlines.labs
LEFT JOIN (
            SELECT  uid,
                    count(*) AS pageviews
            FROM pageviews
            GROUP BY uid
          ) AS number_of_Newsfeed_visits_per_user ON test.uid = number_of_Newsfeed_visits_per_user.uid
WHERE NOT test.labname = 'project1'
GROUP BY test.uid
"""

df_test_the_hypothisis = pd.io.sql.read_sql(aggregated_query_test_the_hypothesis, con)
df_test_the_hypothisis

,uid,delta,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [108]:
df_test_the_hypothisis.corr()

,delta,pageviews
delta,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [ ]:
con.close()